## VOLATILITY STUDY

In the directory, you are given four asset prices historical data at a granularity of one minute (data_challenge.pkl).
The purpose of this study is to analyze the historical volatility of these four assets, and forecast one day ahead volatility to give your trader an indication of what strategies he should use tommorrow. 

# Part 1

1.1) Extract the data from the pickled file, and plot the time series. 

In [ ]:
import pandas as pd

#------------- Extract the data here -------------
df = pd.DataFrame()
#-------------------------------------------------


#------------------Plot data----------------------
df.plot()


1.2) Convert the dataframe to minute returns (percentage, log, ... choose yourself). Then give an overview of each of the descriptive statistics for each of the asset. In particular, provide the first four moments and the 25th, 50th, and 75th quantiles.


In [ ]:
#----------- Convert to returns here -------------

#-------------------------------------------------



#-------- Compute and print the stats here -------

#-------------------------------------------------

1.3.1) Which distribution among: <br>
    - Normal distribution <br>
    - T-student distribution <br>
    - Cauchy distribution <br>
fits the data best? Please back your choice by statistical tests, and describe your results briefly. 

1.3.2) Build a function that plots a theoretical distribution against the empirical distribution of a specific asset. Plot the distribution that you consider the best. 

In [ ]:
#------ Compute and print test results here ------

#-------------------------------------------------

Please, describe briefly the results obtained in this blank space: <br>
--------------------------------------------------------------------------------------------<br>



--------------------------------------------------------------------------------------------<br>

In [ ]:
#------ Define plotting function here ------------
def distribution_plot(returns: pd.Series, distribution: str = 'norm'):
        """
        Empirical distribution is plotted against theoretical one.

        Inputs:
        - returns: pd.Series containing the historical returns of the specific asset. 
        - distribution: string defining the theoretical distribution to fit to data. 

        Returns:
        - plt.plot: Empirical vs Theoretical distribution.
        """

        if distribution == 'norm':
           pass
        elif distribution == 't':
            pass
        elif distribution == 'cauchy':
           pass
        else:
            raise ValueError("Distribution not available. Choose among 'norm', 't', 'cauchy'.")
        
        return 0
#-------------------------------------------------

distribution_plot()

1.4) The following QQ plot tests the empirical distribution of one of the assets against a normal distribution. What can you infer about the tails of the empirical distributions by looking at the plot? <br>
<div style="text-align:center">
    <img src="QQ.png" alt="Distribution Plot" />
</div>

Please, write your answer in this blank space: <br>
--------------------------------------------------------------------------------------------<br>



--------------------------------------------------------------------------------------------<br>

1.5) Are the time series stationary? 

In [ ]:
#------------- Study stationarity here -----------

#-------------------------------------------------

Please, describe briefly the results obtained in this blank space: <br>
--------------------------------------------------------------------------------------------<br>



--------------------------------------------------------------------------------------------<br>

# Part 2

2.1.1) Fit a GARCH(1, 1) model to each of the assets and plot the historical path of volatility for one of the assets. <br>
2.1.2) The trader is asking you what you think volatility will be tomorrow. Forecast the volatility for the day immediately after the last day of the data set and print it.

In [ ]:
#--------- Compute the GARCH model here ----------

#-------------------------------------------------

2.2) Let's suppose you have two way of forecasting volatility for tomorrow: <br>
- 5 minutes realized volatility of today is used as expected value of tomorrow volatility as <br> <br>
        <center>
         $ E[\sigma_{t+1}] = \sigma_{t}^{RV, 5min}$ 
        </center> <br>
- Using the GARCH(1,1) you estimated to forecast tomorrow volatility.

How does your model perform compared to the benchmark of the previous days 5 minute realized volatility? Which one appears to be the most reliable?


In [11]:
#--------- Compare the two methodologies here ----------

